In [ ]:
"""
References to functions in the code:
integer_modulo(x,n) # x mod n
polynomial_modulo(p,n,v) # p mod n
polynomial_division(p,d,v) # p / d, return [quotient, remainder]
def GF(p,q,v)
degree_of_q_polynomials(p,q,v)
find_irreducible_polynomial(p,q,v) # irreducible polynomials of degree q

---- find compositional irreducible polynomials mod 2 (composed with all polynomials in the field)(exclude constants) ----
find_compositional_irreducible_mod2_by_allPoly(irre,p,q,v)

"""

In [2]:
def integer_modulo(x,n):
    if n<=0:
        return "n must be a positive integer"
    r=x
    while r>=n:
        r=r-n
    while r<0:
        r=r+n
    return r

In [3]:
def polynomial_modulo(p,n,v):
    if n<0:
        return "n must be a positive integer"
    newp=0
    for i in range (len(p.coefficients(v))):
        newp=newp+integer_modulo(p.coefficients(v)[i][0],n)*v^p.coefficients(v)[i][1]
    return newp

In [4]:
# p is the dividend; d is the divisor; v is the variable; output[q,r], quotient and remainder
def polynomial_division(p,d,v):
    if p.degree(v)>=1 and d.degree(v)>=1:
        p=p.expand()
        d=d.expand()
        q=0
        r=p
        while p.degree(v)>=d.degree(v):
            L=(p.coefficient(v^p.degree(v))/d.coefficient(v^d.degree(v)))*v^(p.degree(v)-d.degree(v))
            p=p-(L*d).expand()
            q=q+L
        r=p
        return [q,r]
    else:
        if d.degree(v)==0:
            if d==0*v^0:
                return "Invalid input, can't divided by zero!"
            else:
                q=p/d
                r=0
                return[q,r]
        if p.degree(v)==0:
            q=0;
            r=p;
            return[q,r]

In [5]:
def GF(p,q,v):
    poly=[];
    if q!=1:
        for i in range(p):
            new=[i*v^(q-1) + k for k in GF(p,q-1,v)]
            poly=poly+new
    else:
        for i in range(p):
            poly.append(i)
    return poly

In [6]:
def degree_of_q_polynomials(p,q,v):
    if q<1:
        return "q should >= 1"
    field=GF(p,q,v)
    poly=[]
    for i in range(1,p):
        for j in field:
            poly.append(i*v^q+j)
    return poly

In [7]:
#Outputs all the irreducible polynomials in the varible v that with coefficients less than p, and degree of q
def find_irreducible_polynomial(p,q,v):
    field = GF(p,q,v) 
    
    poly=[] # the same as poly = degree_of_q_polynomials(p,q,v), but more efficient
    for i in range(1,p):
        for j in field:
            poly.append(i*v^q+j)
    
    for i in range(p): # remove all constants, which cannot produce a polynomial of degree q
        field.remove(i)
    
    for i in field:
        for j in range( p ** (q - i.degree(v)) - p, len(field)): # polynomial with smaller degree cannot produce a polynomial of degree q
            result = polynomial_modulo((i * field[j]).expand(),p,v)
            if result.degree(v) == q and result in poly: # remove reducible polynomial and avoid multiple removing of the same polynomial
                poly.remove(result) 
    return poly

In [11]:
from itertools import combinations 
def find_compositional_irreducible_mod2_by_allPoly(irre,p,q,v):
    field = GF(p,q,v)
    
    field_of_degree_q = []
    for i in range(1,p):
        for j in field:
            field_of_degree_q.append(i*v^q+j)

    # remove all constants
    field.remove(0)
    field.remove(1)
    
    for i in field: # to compose with i
        
        individual_compo = [] # store all composition polynomials by compose i with one non-constant polynomial in the field
        for j in field:
            new=j.subs(v==i)
            new=polynomial_modulo(new,2,v)
            if new not in individual_compo:
                individual_compo.append(new)
        
        
        compo = [] # store all possible combinations of component composition polynomials
        for j in range(1,len(individual_compo)+1):
            new2=list(combinations(individual_compo, j))
            
            compo=compo+new2
           
        
        
        for j in range(len(compo)):
            new3=0
            
            for k in range(len(compo[j])):
                new3=new3+compo[j][k]
            
            new3=polynomial_modulo(new3,2,v)
            #comment the next line to get compositional polynoamils with respect to no irreducible polnomial
            new3 = polynomial_division(new3,irre,v)[1] # the remainder is a polynomial with degree less than or equal q
            new3 = polynomial_modulo(new3,2,v)
            if new3.degree(v) == q and new3 in field_of_degree_q:
                field_of_degree_q.remove(new3)
    return field_of_degree_q
    

In [14]:
v=var('v')
irre = find_irreducible_polynomial(2,5,v)
for a in irre:
    poly=find_compositional_irreducible_mod2_by_allPoly(a,2,4,v);
    print(poly)

[v^4 + v^3 + v^2, v^4 + v^3 + v^2 + 1]
[v^4 + v^3 + v^2 + v, v^4 + v^3 + v^2 + v + 1]
[]
[v^4 + v^3 + v, v^4 + v^3 + v + 1]
[]
[v^4 + v^3, v^4 + v^3 + 1]


In [15]:
degree_of_q_polynomials(2,4,v)

[v^4,
 v^4 + 1,
 v^4 + v,
 v^4 + v + 1,
 v^4 + v^2,
 v^4 + v^2 + 1,
 v^4 + v^2 + v,
 v^4 + v^2 + v + 1,
 v^4 + v^3,
 v^4 + v^3 + 1,
 v^4 + v^3 + v,
 v^4 + v^3 + v + 1,
 v^4 + v^3 + v^2,
 v^4 + v^3 + v^2 + 1,
 v^4 + v^3 + v^2 + v,
 v^4 + v^3 + v^2 + v + 1]